In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import altair as alt

## Reinventing the Wheel: A DIY approach to Data Analysis and Data Science

The approach of the Reinventing the Wheel (RtW), is to start from a small, sometime not that small, topic, disassemble it into even smaller pieces and then reassembling it traying to mimic the original form. This approach may help in understand better the basic concepts underying and "own" it. Most of the times, when you dismantle an object and rebuild it, you can end up with something that is clunkier or even look ugly. But for sure is something that you own.

In this notebook, I will discuss Box Plots, a method used in descriptive statistics to visually show the [locality, spread and skewness groups of numerical data throug their quartiles](https://en.wikipedia.org/wiki/Box_plot). As a starting point, I will use the [`altair` boxplot](https://altair-viz.github.io/) of a dataset and drill down concepts like median, quartile, IQR, outliers as well as illustrating python and altair procedure often used in Data Analysis.

To illustrate all the concepts, I will use the [Super Store Sales sample data](https://public.tableau.com/app/learn/sample-data) provided by Tableau Public. Some of the ideas I used for this notebook come from the Coursera Course Data Analysis with Tableau, by Tableau Learning Partner.

# Box Plots and Data Distributions

Box plots are a type of visualization that show a statistical summary of selected data. While histograms gives you a graphical understanding about how the data are distributed, and so helpfully indicating if they are evenly distributed, normal or skewed, the advantage of Box Plots is that they provide a visual representation of some of the main data distribution main characteristics, specifically median, quartiles and outliers.

In [2]:
data = pd.read_excel('./data/superstore_data.xlsx')
data.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


From the data, we create a dataframe containing, for each Sub Category, the sum of the sales and visualize the distribution using a boxplot

In [3]:
df_subcategory_sales = (data
 .groupby('Sub-Category', observed=True)
 .agg(
     sales = ('Sales', 'sum'),
 )
).sort_values('sales').reset_index()

box = (alt
       .Chart(df_subcategory_sales)
       .mark_boxplot(size = 40)
       .encode(
           y = alt.Y('sales:Q')
       )
      )

box.properties(
    title = 'Sales by Sub-category box plot',
    width = 100,
    height = 420
)

alt.Chart(...)

Hovering on the newly created Box Plot, Altair provides the following information:
- `Max` of sales: 330007.054
- `Q3` of sales: 203412.733
- `Median` of sales: 114879.9963
- `Q1` of sales: 46673.538
- `Min` of sales: 3024.24

And
- `Upper Whisker` of sales: 330007.054
- `Lower Whisker` of sales: 3024.24

These values aims to show how the data are distributed. But what are their meaning and how can we practically use when analysing the data? `Min` and `Max` are straigthforwardly indicating the minimum and maximum values of the dataset, but what `Median`, `Q1`, `Q3`, `Upper Whisker` and `Lower Whisker` mean and how `altair` calculate it?  

Let's start looking at the concepts of median, quartile and Interquartile Range (IQR) and undersdand their role in the statistical description of a dataset.

Dataset Median, Quartile and Interquartile Range

When analysing dataset, we are dealing with discrete quantities (samples) that might be extracted from continuous distributions, like the height of a person or the distance covered by a starship. When calculating statistical measures of a dataset, like the median, we will always do it on a finite number of data. A way to characterize a dataset, is to divide its elements into parts, and one of the most common way to do it, is dividing it into four sets with the same number of elements each: these are called ***quartiles***.

### Median

> The ***median*** of a set of numbers is the value separating the higher half from the lower half of a data sample, a population, or a probability distribution.

To compute the median of a set of number, we have to consider if the number of its element is odd or even. If the number of data points is odd, the median is the middle point, otherwise, the median is usually defined as the arithmetic mean of the two middle values.

In the following cell we define a `dataset_median` function that takes a pandas series as input, sort it, calculate the midpoint and, based on the fact it is an odd or even number, compute the median of the dataset using the above definition

In [4]:
def dataset_median(dataset):
    """
    input: dataset is a pandas series
    output: returns the median of the dataset
    """
    midpoint = len(dataset)/2
    if midpoint%2 != 0:
        # if the lenths of the dataset is odd, return the middle point 
        return(dataset.sort_values().iloc[int(midpoint)])
    # otherwise, return the arithmetic mean of the two middle points
    return((dataset.sort_values().iloc[int(midpoint)-1]+dataset.sort_values().iloc[int(midpoint)])/2)

In our example, the sum of `Sales` per `Sub-Category` in the Super Store dataset, we have an odd number of elements. This means that the median `sales` is the 8<sup>th</sup> element of the sorter `sales` series.

In [10]:
dataset = df_subcategory_sales['sales']

print("median using the diy function: {}".format(dataset_median(dataset)))
print("median using pandas median   : {}".format(dataset.median()))
## printing the Sub-Category
print("Sub-Category: {}".format(df_subcategory_sales[df_subcategory_sales['sales'] == dataset_median(df_subcategory_sales['sales'])]['Sub-Category'].values[0]))

median using the diy function: 114879.9963
median using pandas median   : 114879.9963
Sub-Category: Bookcases


The following is the altair box-plot of the dataframe `sales` column. As we can see by hoovering on the data points, the median contains the `Bookcases` sub-category. When the dataset length is odd, the median point belongs to the $2$<sup>nd</sup> percentile ($50\%$ of the data)

In [11]:
points = (alt
          .Chart(df_subcategory_sales)
          .mark_point(size = 50, filled=True, opacity=0.8, color = '#953f0a')
          .encode(
              x = alt.X('sales:Q'),
              tooltip = ['Sub-Category:N', 'sales:Q']                        
          )
         )

box = (alt
       .Chart(df_subcategory_sales)
       .mark_boxplot(size = 40)
       .encode(
           x = alt.X('sales:Q')
       )
      )
       

chart = (box + points).properties(
    title = 'Sales by Sub-category box plot',
    width = 620,
    height = 100
)

chart

alt.LayerChart(...)

When we have an odd numer of sample in our dataset, the median also correspond to a specific point of the dataset (in our case is Bookcases).

We now remove the median point from the dataset and compute the new median of the `sales` dataset, now consisting of an odd number of elements.

In [13]:
df_subcategory_without_bookcases = df_subcategory_sales.drop(df_subcategory_sales[df_subcategory_sales['Sub-Category'] == 'Bookcases'].index).reset_index(drop = True)
df_subcategory_without_bookcases.shape[0]

16

Applying the same steps as before, we obtain:

In [14]:
print("median using the diy function: {}".format(dataset_median(df_subcategory_without_bookcases['sales'])))
print("median using pandas median   : {}".format(df_subcategory_without_bookcases['sales'].median()))

# As we have an even number of elements, there is no sub category that is the median of the dataset
# print("Sub-Category: {}".format(df_subcategory_without_bookcases[df_subcategory_without_bookcases['sales'] == dataset_median(df_subcategory_without_bookcases['sales'])]['Sub-Category'].values[0]))

median using the diy function: 128530.0955
median using pandas median   : 128530.0955


Plotting the box plot, we can see that now the median is between the two middle points.

In [23]:
points = (alt
          .Chart(df_subcategory_without_bookcases)
          .mark_point(size = 50, filled=True, opacity=0.8, color = '#953f0a')
          .encode(
              x = alt.X('sales:Q'),
              tooltip = ['Sub-Category:N', 'sales:Q']                        
          )
         )

box = (alt
       .Chart(df_subcategory_without_bookcases)
       .mark_boxplot(size = 40)
       .encode(
           x = alt.X('sales:Q')
       )
      )
       

chart = (box + points).properties(
    title = 'Sales by Sub-category box plot',
    width = 620,
    height = 100
)

chart

alt.LayerChart(...)

In [24]:
np.sqrt(0.85)

0.9219544457292888

In [27]:
(52.9-4.3)/2

24.3

In [30]:
(144+72)/2

108.0

In [29]:
108418/12226

8.867822672991984

In [26]:
223843/(719047+836154+741999)

0.09744166811770852

In [22]:
values =  [0, 3, 4.4, 4.5, 4.6, 5, 7]
df = pd.DataFrame({'x': values})

points = alt.Chart(df).mark_circle(color='black', size=120).encode(
    x=alt.X('x:Q', scale=alt.Scale(zero=False)),
)

boxplot = alt.Chart(df).mark_boxplot(ticks=True, extent=1.5, outliers=True).encode(
    x='x:Q',
)

iqr = alt.Chart(df).mark_rect(color='lime').encode(
    x='q1(x):Q',
    x2='q3(x):Q'
)

whiskers = alt.Chart(df).mark_rect(color='orange').transform_joinaggregate(
    q1='q1(x)',
    q3='q3(x)',
).transform_calculate(
    iqr='datum.q3 - datum.q1'
).transform_filter(
    # VL concatenates these strings so we can split
    # them on two lines to improve readability
    'datum.x < (datum.q3 + datum.iqr * 1.5)'
    '&& datum.x > (datum.q1 - datum.iqr * 1.5)'
).encode(
    x='min(x)',
    x2='max(x)',
)

minmax = alt.Chart(df).mark_rect(color='red').transform_aggregate(
    xmin='min(x)',
    xmax='max(x)'
).encode(
    x='xmin:Q',
    x2='xmax:Q',
).properties(width=1000)


((boxplot + points) & (minmax + whiskers + iqr + points)).resolve_scale(x='shared')

alt.VConcatChart(...)

In this plot, the median value 

In [ ]:
df_subcategory_without_bookcases['sales'].median()

In [ ]:
df_subcategory_sales

In [ ]:
dataset_median(new_dataset)

In [ ]:
dataset.median()

In [ ]:
df_subcategory_sales = (data
 .groupby('Sub-Category', observed=True)
 .agg(
     sales = ('Sales', 'sum'),
 ).reset_index()
)

dataset = df_subcategory_sales['sales']

midpoint = len(dataset)/2
if midpoint%2 != 0:
    print(dataset.sort_values().iloc[int(midpoint)])
else:
    print((dataset.sort_values().iloc[int(midpoint)]+dataset.sort_values().iloc[int(midpoint)+1])/2)

# dataset.sort_values()

In [ ]:
new_dataset = dataset.drop(labels=dataset[dataset.values == 114879.9963].index)

In [ ]:
new_dataset.median()

In [ ]:
midpoint = len(new_dataset)/2
if midpoint%2 != 0:
    print(new_dataset.sort_values().iloc[int(midpoint)])
else:
    print((new_dataset.sort_values().iloc[int(midpoint)-1]+new_dataset.sort_values().iloc[int(midpoint)])/2)

In [ ]:
dataset.sort_values().iloc[1]

In [ ]:
median = df_subcategory_sales['sales'].median()
print("sales median as computed by numpy: {}".format(median))


df_subcategory_sales.iloc[8]

In [ ]:
def median(dataset):
    len_data = df_subcategory_sales.shape[0]
    
    if (len_data % 2) != 0:
        mid_point_index = len_data //2
        print(df_subcategory_sales.iloc[mid_point_index])

In [ ]:
dataset = df_subcategory_sales['sales']
dataset.sort_values()

In [ ]:
df_subcategory_sales.sort_values('sales').reset_index(drop=True).iloc[3]

In statistics, ***quantiles*** are particular points dividing a sample into equally sized, adjacent subgroups. As an example, the median is a quantiles, so that exactly half of the data is lower than the median and half of the data is above the median.
***Quartiles*** divide the distribution into four equal part.

In our example, we are considering the sales points from the different Sub Categories. To determine which data point belongs to the right quartile, se sort the values and divide the array into 4 equal parts:

The middle point of the `sales` column is Bookcases, 114879.9963: this value represent the median of the dataset. Now, we break down it further into 4 equal parts and obtain:
- 1st Quartile: `Fastners`, `Labels`, `Envelops` and `Art`
- 2nd Quartile: `Supplies`, `Paper`, `Furnishing` and `Appliances`
- Median: `Bookcases`
- 3rd Quartile: `Copiers`, `Accessories`, `Machines` and `Binders`
- 4th Quartile: `Tables`, `Storage`, `Chairs` and `Phones`

`pandas` and `numpy` both offer methods to easily compute quartile, but to better understand it, let's reinvent the wheel and define a Python function that do the same. To do it, we will go back to the very definition of quartile and focus on the standard definition about the percentages (0.25, 0.5, 0.6

In [ ]:
df_subcategory_sales[df_subcategory_sales['sales'] < df_subcategory_sales['sales'].quantile(0.25)]

In [ ]:
df_subcategory_sales[(df_subcategory_sales['sales'] >= df_subcategory_sales['sales'].quantile(0.25)) & (df_subcategory_sales['sales'] < df_subcategory_sales['sales'].median())]

To make it more clear, let's plot the box plot and the associated points

In [ ]:
points = (alt
          .Chart(df_subcategory_sales)
          .mark_point(size = 50, filled=True, opacity=0.8, color = '#953f0a')
          .encode(
              x = alt.X('sales:Q'),
              tooltip = ['Sub-Category:N', 'sales:Q']                        
          )
          # .configure_mark(
          #     opacity=0.8,
          #     color='#953f0a'
          # )
         )

box = (alt
       .Chart(df_subcategory_sales)
       .mark_boxplot(size = 40)
       .encode(
           x = alt.X('sales:Q')
       )
      )
       

chart = (box + points).properties(
    title = 'Sales by Sub-category box plot',
    width = 620,
    height = 100
)

chart

In [ ]:
df_test = df_subcategory_sales[df_subcategory_sales['Sub-Category'] != 'Bookcases'].reset_index(drop=True)
df_test

In [ ]:
points = (alt
          .Chart(df_test)
          .mark_point(size = 50, filled=True, opacity=0.8, color = '#953f0a')
          .encode(
              x = alt.X('sales:Q'),
              tooltip = ['Sub-Category:N', 'sales:Q']                        
          )
         )

box = (alt
       .Chart(df_test)
       .mark_boxplot(size = 40)
       .encode(
           x = alt.X('sales:Q')
       )
      )
       

chart = (box + points).properties(
    title = 'Sales by Sub-category box plot',
    width = 620,
    height = 100
)

chart

## IQR - Interquartile Range

In our example, the dataset consist of points that are quete close each other, meaning that they do not particularly spread all over the possible values. Let's now focus our analyisis on the same Sales dataset but we now want to consider the sales only in the `Central` region.

In [ ]:
df_central_sales = (data[data['Region'] == 'Central']
                    .groupby('Sub-Category')
                    .agg(
                        sales = ('Sales','sum')
                    )
                   ).sort_values('sales').reset_index()


points = (alt
          .Chart(df_central_sales)
          .mark_point(size = 50, filled=True, opacity=0.8, color = '#953f0a')
          .encode(
              y = alt.Y('sales:Q'),
              tooltip = ['Sub-Category:N', 'sales:Q']                        
          )

         )

box = (alt
       .Chart(df_central_sales)
       .mark_boxplot(size = 40)
       .encode(
           y = alt.Y('sales:Q')
       )
      )
       

chart = (box + points).properties(
    title = 'Sales by Sub-category in Central Region box plot',
    width = 100,
    height = 420
)

chart

In [ ]:
df_subcategory_profit = (data
 .groupby('Sub-Category', observed=True)
 .agg(
     profit = ('Profit', 'sum'),
 )
).reset_index().sort_values('profit', ascending = False)

box = (alt
       .Chart(df_subcategory_profit)
       .mark_boxplot(size = 40)
       .encode(
           y = alt.Y('profit:Q')
       )
      )

box.properties(
    title = 'Profit by Sub-category box plot',
    width = 100,
    height = 420
)

In [ ]:
df_subcategory_profit

In [ ]:
points = (alt
          .Chart(df_subcategory_profit)
          .mark_point(size = 50, filled=True, opacity=0.8, color = '#953f0a')
          .encode(
              y = alt.Y('profit:Q'),
              tooltip = ['Sub-Category:N', 'profit:Q']                        
          )
          # .configure_mark(
          #     opacity=0.8,
          #     color='#953f0a'
          # )
         )

box = (alt
       .Chart(df_subcategory_profit)
       .mark_boxplot(size = 40)
       .encode(
           y = alt.Y('profit:Q')
       )
      )
       

chart = (box + points).properties(
    title = 'Profit by Sub-category box plot',
    width = 100,
    height = 420
)

chart

The Interquartile Range is an important measure of statistical dispersion and its definition is quite basic: is the difference between the third and first quartile. 

$$
IQR = Q_{3}- Q{1}
$$

In [ ]:
df_subcategory_sales['sales'].quantile(0.75) - df_subcategory_sales['sales'].quantile(0.25)

The interquartile range plays an important role when we want to identify points in the dataset that are potentially outliers, i.e. points quite far away from the distribution. In a boxplot, 

In statistics, 

In [ ]:
df_subcategory_sales['sales'].min()

In [ ]:
base = (alt
 .Chart(df_subcategory_sales)
#  .mark_point()
 .encode(
#     x = 'Sub-Category:',
     y = 'sales'
 ).properties(
    width=200
 )
)

base.mark_point() + base.mark_boxplot() 

In [ ]:
# Create the dataframe
dataX = {
    'Sub-Category': ['Phones', 'Chairs', 'Storage', 'Tables', 'Binders', 'Machines', 
                     'Accessories', 'Copiers', 'Bookcases', 'Appliances'],
    'Sales': [330007.0540, 328449.1030, 223843.6080, 206965.5320, 203412.7330, 
              189238.6310, 167380.3180, 149528.0300, 114879.9963, 107532.1610]
}

dfX = pd.DataFrame(dataX)

# Create the Altair scatter plot
chart = alt.Chart(dfX).mark_point().encode(
    x=alt.X('Sub-Category:N', sort='-y', title='Sub-Category'),
    y=alt.Y('Sales:Q', title='Sales'),
    tooltip=['Sub-Category:N', 'Sales:Q']  # Add tooltips
).properties(
    title='Sales by Sub-Category',
    width=500,
    height=300
)

chart.display()

In [ ]:
np.round(df_subcategory_sales['sales'].median(),0)

In [ ]:
np.quantile(df_subcategory_sales['sales'], 0.75)

In [ ]:
data.columns

In [ ]:
data.Region.unique()

In [ ]:
df_region_subcat = (data
                    .groupby(['Region', 'Sub-Category'])
                    .agg(
                        sales = ('Sales','sum'),
                        profit = ('Profit', 'sum'),
                        quantity = ('Quantity', 'sum')
                    )
                   ).reset_index()

df_region_subcat.head(2)

In [ ]:
(alt
 .Chart(df_region_subcat)
 .mark_boxplot(ticks=True, size=15)
 .encode(
     x = 'Region:O',
     y = 'sales:Q',
 )
 .properties(
    width=200
 )
 .configure_view(
    stroke=None
 )
)

In [ ]:
x = [1,2,3,4,5] 
y = [1,4,9,16,25]

plt.plot(x, y)
plt.title('Square Numbers')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

In [ ]:
df = pd.DataFrame({'x': [1, 2, 3, 4, 5], 'y': [1, 4, 9, 16, 25]})

chart = (alt
         .Chart(df)
         .mark_line()
         .encode(
             x='x',
             y='y'
         ).properties(
             title='Square Numbers'
         )
        )
chart